In [1]:
import sys
sys.path.append('../scripts')

import requests
import zipfile
import xml.etree.ElementTree as ET
import os
import pickle
from utils import download, cleanseTransWide, convColNm, cleanSalesData, cleanCmpnyNm, catOwn, catOwn2
from selenium import webdriver
import chromedriver_autoinstaller
import numpy as np
import pandas as pd

In [2]:
chromedriver_autoinstaller.install(path="../utils/")

'/Users/choidamian/Chaebol/utils/101/chromedriver'

# Download Company Code File

In [317]:
class Api:
    
    def __init__(self, apiKey):
        
        self.apiKey = apiKey
        self.corpCode = None

    def getCorpCode(self, path):

        url = "https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=" + self.apiKey
        download(url, path)

        with zipfile.ZipFile(path, 'r') as zipObj:
            fileList = zipObj.namelist()
            target = fileList[0]
        
        loc = os.path.normpath(path)
        loc = loc.split(os.sep)
        loc = '/'.join(loc[:-1])
        
        with zipfile.ZipFile(path, 'r') as zipRef:
            zipRef.extractall(loc)

        self.corpCode = loc + '/' + target
        print(f'Corporation Code File downloaded to {self.corpCode}')

    def findCorpCode(self, corpNm):
        
        if not self.corpCode:
            print('No corporation code file found. Downloading the code file...')
            self.getCorpCode('../data/raw/opendart/corpCode/CORPCODE.zip')

        xml = ET.parse(self.corpCode)
        root = xml.getroot()

        lists = root.findall('list')

        codeList = []
        for list in lists:
            if (list.find('corp_name').text == corpNm):
                codeList.append(list.find('corp_code').text)
                
        return codeList

    def findCorpCode(self, corpNm):
        
        if not self.corpCode:
            print('No corporation code file found. Downloading the code file...')
            self.getCorpCode('../data/raw/opendart/corpCode/CORPCODE.zip')

        xml = ET.parse(self.corpCode)
        root = xml.getroot()

        lists = root.findall('list')

        codeList = []
        for list in lists:
            if (list.find('corp_name').text == corpNm):
                codeList.append(list.find('corp_code').text)
        return codeList

    def findRceptNum(self, corpNm, year):

        codeList = self.findCorpCode(corpNm)
        pblntfTy = 'j'
        pblntfDetailTy = "J004"
        pgCnt = "50"
        bgnDe = str(year) + '0501'
        endDe = str(year+1) + '0430'

        # find the firm that submitted reports to FTC whose name is <firmname>
        for corpCode in codeList:
            url = "https://opendart.fss.or.kr/api/list.json?corp_code=" + corpCode + "&crtfc_key=" + self.apiKey \
            + "&bgn_de=" + bgnDe + "&end_de=" + endDe + "&last_reprt_at=Y" + "&pblntf_ty=" + pblntfTy + \
            "&pblntf_detail_ty=" + pblntfDetailTy + "&page_count=" + pgCnt
            res = requests.get(url)
            content = res.json()
            if 'list' in content:
                break
        
        else:
            raise ValueError(f"{corpNm}: no report found for {bgnDe} ~ {endDe}")


        # find all the reports whose names contain the string '대규모기업집단현황공시[연1회공시및1/4분기용' and save as a list
        # and return rcept_no of the last report in the list
        receptNums = [report['rcept_no'] for report in content['list'] if ('대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)]' \
                        in report['report_nm'])]
        
        if (corpNm in ['아모레퍼시픽그룹', '아이엠엠인베스트먼트'] and year == 2021):
            receptNums = [receptNums[0]] # select the last report received
        
        assert len(receptNums) >= 1, f'{corpNm}: no 대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)] found for {bgnDe} ~ {endDe}'
        assert len(receptNums) <= 1, f'{corpNm}: Too many 대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)] report found'

        return receptNums[0]

    def getSharesURL(self, corpNm, year):

        rceptNum = self.findRceptNum(corpNm, year)
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + rceptNum
        driver.get(url)

        # In the navigation menu click the link that contains the string '소유지분현황'
        driver.find_element_by_partial_link_text('소유지분현황').click()

        # get the url of the '소유지분현황' page and return it
        sharesURL = driver.find_element_by_id('ifrm').get_attribute('src')
        
        # quit
        driver.quit()

        return sharesURL

    def getSharesTable(self, corpNm, year):
        
        # get shares url
        url = self.getSharesURL(corpNm, year)
        
        # parse html tables which includes string data '동일인'(the same person in Korean) with thousands delimiter ',' 
        tables = pd.read_html(url, match='동일인', thousands=',')
        
        # convert column names
        for table in tables:
            if len(table.columns) != 12:
                print(table.head())
                
            if len(table.columns) == 11 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(4, '금융회사', np.nan)
            elif len(table.columns) == 10 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(2, '동일인과의관계', '동일인측')
                table.insert(4, '동일인과의관계3', '계열회사(국내+해외)')
            elif len(table.columns) == 9 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(4, '금융회사', np.nan)
                table.insert(8, '우선주주식수', np.nan)
                table.insert(9, '우선주지분율', np.nan)
                
            table.columns = ['금융회사', '소속회사명', '동일인과의관계', '동일인과의관계2', '동일인과의관계3', '성명', \
                             '보통주주식수', '보통주지분율', '우선주주식수', '우선주지분율', '합계주식수', '합계지분율']
            
        # concat tables
        df = pd.concat(tables)
        df.reset_index(inplace=True)
        df.drop(columns=['index'], inplace=True)
        df = df[df['소속회사명'] != '소속회사명']
        
        # data cleaning
        clean(df['소속회사명'])
        clean(df['성명'])
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace(' ', '', regex=True)
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace('⑧', '', regex=True)
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace('⑨', '', regex=True)
        df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
        df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
        df['합계주식수'] = df['합계주식수'].copy().astype(float)

        # save the data in the after_cleansing folder
        if not os.path.exists(f'../data/ownership-status/raw/{year}'):
            os.makedirs(f'../data/ownership-status/raw/{year}')
        df.to_excel(f'../data/ownership-status/raw/{year}/{corpNm}.xlsx', index=False)

        return df
    
    def getSharesTableAll(self, year):
        
        dfAppnGroupSttus = pd.read_excel(f'../data/grpSumry/appnGroupSttus/appnGroupSttus{year}.xlsx')
        dfAppnGroupSttus['repreCmpny'].replace('에스케이', 'SK', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('엘지', 'LG', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('지에스', 'GS', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('한진칼', '대한항공', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('씨제이', 'CJ', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('엘에스', 'LS', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('네이버', 'NAVER', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('에이치디씨', 'HDC', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('디엘', 'DL', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('에쓰-오일', 'S-Oil', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('오씨아이', 'OCI', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('한국투자금융지주', '한국금융지주', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('디비아이엔씨', 'DB', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('에이케이홀딩스', 'AK홀딩스', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('현대해상화재보험', '현대해상', regex=True, inplace=True)
        clean(dfAppnGroupSttus)
        
        with open(f'../data/utils/gNonChaebol{year}.pkl', 'rb') as f:
            gNonChaebol = pickle.load(f)
        
        for cmpny in dfAppnGroupSttus['repreCmpny']:
            if cmpny in gNonChaebol['cmpny']:
                continue
            dfShares = api.getSharesTable(cmpny, year)
            print("="* 15 + cmpny + "=" * 15)
            print(dfShares.head())
            print(dfShares.tail())
            print(dfShares['동일인과의관계'].unique())
            print(dfShares['동일인과의관계2'].unique())
            print(dfShares['동일인과의관계3'].unique())
            print("="* 15 + cmpny + "=" * 15)        

In [323]:
dfAppnGroupSttus21 = pd.read_excel('../data/grpSumry/appnGroupSttus/appnGroupSttus2021.xlsx')
dfAppnGroupSttus21['repreCmpny'].replace('에스케이', 'SK', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('엘지', 'LG', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('지에스', 'GS', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('한진칼', '대한항공', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('씨제이', 'CJ', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('엘에스', 'LS', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('네이버', 'NAVER', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에이치디씨', 'HDC', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('디엘', 'DL', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에쓰-오일', 'S-Oil', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('오씨아이', 'OCI', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('한국투자금융지주', '한국금융지주', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('디비아이엔씨', 'DB', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에이케이홀딩스', 'AK홀딩스', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('현대해상화재보험', '현대해상', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에이치엠엠', 'HMM', regex=True, inplace=True)
clean(dfAppnGroupSttus21)
dfAppnGroupSttus21.head(50)

,unityGrupNm,unityGrupCode,smerNm,repreCmpny,sumCmpnyCo,invstmntLmtt,entrprsCl
0,삼성,K1000032,이재용,삼성전자,59,상호출자제한집단,일반
1,현대자동차,K3000027,정의선,현대자동차,53,상호출자제한집단,일반
2,에스케이,K1000050,최태원,SK,148,상호출자제한집단,일반
3,엘지,K1000051,구광모,LG,70,상호출자제한집단,일반
4,롯데,K1000027,신동빈,롯데지주,86,상호출자제한집단,일반
5,포스코,K1000064,포스코,포스코홀딩스,33,상호출자제한집단,일반
6,한화,K1000082,김승연,한화,83,상호출자제한집단,일반
7,지에스,K3000076,허창수,GS,80,상호출자제한집단,일반
8,현대중공업,K3000052,정몽준,한국조선해양,33,상호출자제한집단,일반
9,농협,K3000091,농업협동조합중앙회,농업협동조합중앙회,58,상호출자제한집단,일반


In [324]:
gNonChaebol21 = {'grp': ['포스코', '농협', '케이티', '에쓰-오일', '대우조선해양', \
                         '케이티앤지', '대우건설', '에이치엠엠', '한국지엠', '쿠팡', '한국항공우주산업'],
                 'cmpny': ['포스코홀딩스', '농업협동조합중앙회', '케이티', 'S-Oil', '대우조선해양', \
                           '케이티앤지', '대우건설', 'HMM', '한국지엠', '쿠팡', '한국항공우주산업']}

In [320]:
with open('../data/utils/gNonChaebol21.pkl', 'wb') as f:
    pickle.dump(gNonChaebol21, f)

In [325]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
for cmpny in dfAppnGroupSttus21['repreCmpny'][47:]:
    if cmpny in gNonChaebol21['cmpny']:
        continue
    df = api.getSharesTable(cmpny, 2021)
    print("="* 15 + cmpny + "=" * 15)
    print(df.head())
    print(df.tail())
    print(df['동일인과의관계'].unique())
    print(df['동일인과의관계2'].unique())
    print(df['동일인과의관계3'].unique())
    print("="* 15 + cmpny + "=" * 15)

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
===============태광산업===============
    금융회사 소속회사명 동일인과의관계 동일인과의관계2 동일인과의관계3   성명    보통주주식수  보통주지분율 우선주주식수 우선주지분율  \
0  비금융회사  태광산업    동일인측      동일인      동일인  이호진  328189.0   29.48    NaN    NaN   
1  비금융회사  태광산업    동일인측       친족   혈족2~4촌  이원준   83370.0    7.49    NaN    NaN   
2  비금융회사  태광산업    동일인측       친족   혈족2~4촌  이동준    8918.0    0.80    NaN    NaN   
3  비금융회사  태광산업    동일인측       친족   혈족2~4촌  이태준    8982.0    0.81    NaN    NaN   
4  비금융회사  태광산업    동일인측       친족   혈족5~6촌  NaN       NaN     NaN    NaN    NaN   

      합계주식수  합계지분율  
0  328189.0  29.48  
1   83370.0   7.49  
2    8918.0   0.80  
3    8982.0   0.81  
4       NaN    NaN  
     금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3      성명     보통주주식수  \
291  금융회사  예가람저축은행    동일인측         계열회사         계열회사  흥국생명보험   553018.0   
292  금융회사  예가람저축은행    동일인측       동일인측합계       동일인측합계     NaN  

===============삼천리===============
    금융회사 소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3           성명  보통주주식수 보통주지분율 우선주주식수  \
0  비금융회사   삼천리    동일인측      동일인       동일인          이만득  338390   8.34      -   
1  비금융회사   삼천리    동일인측       친족  배우자/혈족1촌          이은희   27045   0.67      -   
2  비금융회사   삼천리    동일인측       친족  배우자/혈족1촌          이은남   27045   0.67      -   
3  비금융회사   삼천리    동일인측       친족  배우자/혈족1촌  이은선YiEunSun   27045   0.67    NaN   
4  비금융회사   삼천리    동일인측       친족    혈족2~4촌          이은백  372070   9.18      -   

  우선주지분율   합계주식수 합계지분율  
0      -  338390  8.34  
1      -   27045  0.67  
2      -   27045  0.67  
3      -   27045  0.67  
4      -  372070  9.18  
     금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3           성명     보통주주식수  \
634  금융회사  아네모이제2호     동일인     기타동일인관련자     기타동일인관련자            -          -   
635  금융회사  아네모이제2호     동일인       동일인측합계       동일인측합계          NaN  465851647   
636  금융회사  아네모이제2호    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주            -          -   
637  금융회사  아네모이

===============반도홀딩스===============
    금융회사  소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3   성명   보통주주식수  보통주지분율 우선주주식수  \
0  비금융회사  반도홀딩스    동일인측      동일인       동일인  권홍사  1621395  69.61%      -   
1  비금융회사  반도홀딩스    동일인측       친족  배우자/혈족1촌  권재현   700000  30.06%      -   
2  비금융회사  반도홀딩스    동일인측       친족  배우자/혈족1촌  유성애     5869   0.25%      -   
3  비금융회사  반도홀딩스    동일인측       친족  배우자/혈족1촌  권보라     1761   0.08%      -   
4  비금융회사  반도홀딩스    동일인측       친족    혈족2~4촌    -        -       -      -   

  우선주지분율    합계주식수   합계지분율  
0      -  1621395  69.61%  
1      -   700000  30.06%  
2      -     5869   0.25%  
3      -     1761   0.08%  
4      -        -       -  
      금융회사   소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3 성명 보통주주식수   보통주지분율  \
641  비금융회사  더스타아시아    동일인측       동일인측합계       동일인측합계  -  30000   75.00%   
642  비금융회사  더스타아시아    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주  -      -        -   
643  비금융회사  더스타아시아    기타주주           기타           기타  /  10000   25.00%   
644  비금융회사  더스타아시아      총계           총계      

===============엠디엠===============
    금융회사 소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3   성명  보통주주식수  보통주지분율 우선주주식수 우선주지분율  \
0  비금융회사   엠디엠    동일인측      동일인       동일인  문주현  190000  95.00%      -      -   
1  비금융회사   엠디엠    동일인측       친족  배우자/혈족1촌  민혜정   10000   5.00%      -      -   
2  비금융회사   엠디엠    동일인측       친족    혈족2~4촌    -       -       -      -      -   
3  비금융회사   엠디엠    동일인측       친족    혈족5~6촌    -       -       -      -      -   
4  비금융회사   엠디엠    동일인측       친족    인척4촌이내    -       -       -      -      -   

    합계주식수   합계지분율  
0  190000  95.00%  
1   10000   5.00%  
2       -       -  
3       -       -  
4       -       -  
     금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3 성명    보통주주식수 보통주지분율  \
290  금융회사  한국자산캐피탈    동일인측     기타동일인관련자     기타동일인관련자  -         -      -   
291  금융회사  한국자산캐피탈    동일인측       동일인측합계       동일인측합계  /  20000000   100%   
292  금융회사  한국자산캐피탈    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주  -         -      -   
293  금융회사  한국자산캐피탈    기타주주           기타           기타  /      

# Ownership Status Data Cleansing

## 태영건설

In [281]:
df = pd.read_excel('../data/ownership-status/raw/2021/티와이홀딩스.xlsx')
df.head()

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율
0,비금융회사,티와이홀딩스,동일인측,동일인,동일인,윤세영,245424,0.61%,-,-,245424,0.59%
1,비금융회사,티와이홀딩스,동일인측,친족,배우자/혈족1촌,윤석민,12827810,31.93%,-,-,12827810,30.96%
2,비금융회사,티와이홀딩스,동일인측,친족,인척4촌이내,이상희,1128953,2.81%,-,-,1128953,2.73%
3,비금융회사,티와이홀딩스,동일인측,친족,친족합계,/,14202187,35.35%,-,-,14202187,34.28%
4,비금융회사,티와이홀딩스,동일인측,비영리법인,비영리법인,서암윤세영재단,2830311,7.05%,-,-,2830311,6.83%


In [282]:
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'][df['합계주식수'].str.contains('\D+', na=False, regex=True)]

257    " 해당사항 없음 "
Name: 합계주식수, dtype: object

In [283]:
# remove " 해당사항 없음 "
df['합계주식수'].replace('\D+', '', regex=True, inplace=True)

In [284]:
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [285]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
df.head()

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
0,비금융회사,티와이홀딩스,동일인측,동일인,동일인,윤세영,245424,0.61%,-,-,245424.0,0.59%,0.0,0.0
1,비금융회사,티와이홀딩스,동일인측,친족,배우자/혈족1촌,윤석민,12827810,31.93%,-,-,12827810.0,30.96%,NaN,NaN
2,비금융회사,티와이홀딩스,동일인측,친족,인척4촌이내,이상희,1128953,2.81%,-,-,1128953.0,2.73%,NaN,NaN
3,비금융회사,티와이홀딩스,동일인측,친족,친족합계,/,14202187,35.35%,-,-,14202187.0,34.28%,0.0,0.0
4,비금융회사,티와이홀딩스,동일인측,비영리법인,비영리법인,서암윤세영재단,2830311,7.05%,-,-,2830311.0,6.83%,NaN,0.0


In [286]:
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfShares.head()

,소속회사명,주식수총계
8,티와이홀딩스,41426953.0
12,군포복합개발피에프브이,5000000.0
17,디엠씨미디어,465920.0
21,리앤에스스포츠,500000.0
26,무노스,76311.0


In [287]:
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfMrg.head()

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2,주식수총계,shares
0,비금융회사,티와이홀딩스,동일인측,동일인,동일인,윤세영,245424,0.61%,-,-,245424.0,0.59%,0.0,0.0,41426953.0,0.005924
1,비금융회사,티와이홀딩스,동일인측,친족,배우자/혈족1촌,윤석민,12827810,31.93%,-,-,12827810.0,30.96%,NaN,NaN,41426953.0,0.309649
2,비금융회사,티와이홀딩스,동일인측,친족,인척4촌이내,이상희,1128953,2.81%,-,-,1128953.0,2.73%,NaN,NaN,41426953.0,0.027252
3,비금융회사,티와이홀딩스,동일인측,친족,친족합계,/,14202187,35.35%,-,-,14202187.0,34.28%,0.0,0.0,41426953.0,0.342825
4,비금융회사,티와이홀딩스,동일인측,비영리법인,비영리법인,서암윤세영재단,2830311,7.05%,-,-,2830311.0,6.83%,NaN,0.0,41426953.0,0.068321


In [288]:
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn

,소속회사명,ownCat,shares
1,걸포도시개발자산과리,99.0,0.840000
3,경산에코에너지,99.0,0.500000
5,군포복합개발피에프브이,99.0,0.646000
7,네오시티,99.0,0.310000
11,디엠씨미디어,99.0,0.085852
14,무노스,99.0,0.399955
16,문고리닷컴,99.0,0.202000
18,부산바이오에너지,99.0,0.800000
19,블루원,0.0,0.122640
23,삼계개발,99.0,0.490000


In [289]:
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfOwn2

,소속회사명,ownCat,shares
1,걸포도시개발자산과리,99.0,0.840000
3,경산에코에너지,99.0,0.500000
5,군포복합개발피에프브이,99.0,0.646000
7,네오시티,99.0,0.310000
10,디엠씨미디어,0.0,0.373626
12,디엠씨미디어,99.0,0.085852
13,리앤에스스포츠,0.0,0.300000
15,무노스,0.0,0.000026
17,무노스,99.0,0.399955
18,문고리닷컴,0.0,0.198000


In [290]:
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfMrgd

,소속회사명,ownCat,own,own2,ownername
0,걸포도시개발자산과리,99.0,0.840000,0.840000,NaN
1,경산에코에너지,99.0,0.500000,0.500000,NaN
2,군포복합개발피에프브이,99.0,0.646000,0.646000,NaN
3,네오시티,99.0,0.310000,0.310000,NaN
4,디엠씨미디어,99.0,0.085852,0.085852,NaN
5,무노스,99.0,0.399955,0.399955,NaN
6,문고리닷컴,99.0,0.202000,0.202000,NaN
7,부산바이오에너지,99.0,0.800000,0.800000,NaN
8,블루원,0.0,0.122640,0.122640,NaN
9,삼계개발,99.0,0.490000,0.490000,NaN


In [167]:
dfMrgd[dfMrgd['소속회사명']== '디엠씨미디어']

,소속회사명,ownCat,own,own2,ownername
3,디엠씨미디어,99.0,1.085852,1.085852,NaN
40,디엠씨미디어,0.0,NaN,0.373626,NaN


In [291]:
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn1

,소속회사명,ownCat,own,own2,ownername
9,군포복합개발피에프브이,1.0,0.354000,0.354000,태영건설
14,디엠씨미디어,1.0,0.540522,0.540522,에스비에스
19,리앤에스스포츠,1.0,0.700000,0.700000,SBS미디어넷
23,무노스,1.0,0.600018,0.600018,SBS미디어넷
28,문고리닷컴,1.0,0.600000,0.600000,에스비에스네오파트너스
...,...,...,...,...,...
244,창원복합행정타운개발,1.0,0.510000,0.510000,태영건설
248,천안풍세일반산업단지,1.0,0.750001,0.750001,태영건설
252,티와이신경주주택개발,1.0,1.000000,1.000000,태영건설
256,포천바이오에너지,1.0,0.510000,0.510000,태영건설


In [292]:
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']

In [293]:
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.head()

,firmname,type,own,own2,ownername
0,걸포도시개발자산과리,1.0,0.580,0.580,태영건설
1,걸포도시개발자산과리,99.0,0.840,0.840,NaN
2,경산에코에너지,1.0,0.500,0.500,태영건설
3,경산에코에너지,99.0,0.500,0.500,NaN
4,군포복합개발피에프브이,1.0,0.354,0.354,태영건설


In [294]:
dfOwn.groupby(['firmname'])['own2'].sum()

firmname
걸포도시개발자산과리     1.420000
경산에코에너지        1.000000
군포복합개발피에프브이    1.000000
네오시티           1.000000
대구남부에이엠씨       1.000000
대동산업단지         1.000000
디엠씨미디어         1.000000
리앤에스스포츠        1.000000
무노스            1.000000
문고리닷컴          1.000000
부산바이오에너지       1.400000
블루원            1.000000
블루원레저          1.000000
삼계개발           1.000000
센트로            1.000000
스마트미디어렙        1.000000
스튜디오에스         1.000000
신경주역세권공영개발     1.000000
신경주지역개발        1.754386
양주동서도로         1.000000
에스비에스          1.000000
에스비에스네오파트너스    1.000000
에스비에스디지털뉴스랩    1.000000
에스비에스미디어넷      1.000000
에스비에스미디어홀딩스    1.000000
에스비에스바이아컴      1.000000
에스비에스아이앤엠      1.000000
에스비에스에이앤티      1.000000
에스비에스엠앤씨       1.000000
에스비에스콘텐츠허브     1.000000
에스비에스플러스       1.000000
에코시스템          1.000000
에코시티           1.000000
에코시티개발         1.000000
엠시에타           1.000000
엠시에타개발         1.000000
여수엑스포환경        1.000000
여천이피에스         1.000000
유니시티           1.000000
이너시티개발         1.000000
인제스피디움         1.000000
지엠파크   

In [295]:
dfOwn[dfOwn['firmname'] == '걸포도시개발자산과리']

,firmname,type,own,own2,ownername
0,걸포도시개발자산과리,1.0,0.58,0.58,태영건설
1,걸포도시개발자산과리,99.0,0.84,0.84,NaN


In [296]:
dfOwn.iloc[0, 0] = '걸포도시개발자산관리'
dfOwn.iloc[1, 0] = '걸포도시개발자산관리'
dfOwn.iloc[1, 2] = 0.42
dfOwn.iloc[1, 3] = 0.42
dfOwn[dfOwn['firmname'] == '걸포도시개발자산관리']

,firmname,type,own,own2,ownername
0,걸포도시개발자산관리,1.0,0.58,0.58,태영건설
1,걸포도시개발자산관리,99.0,0.42,0.42,NaN


In [297]:
dfOwn[dfOwn['firmname'] == '신경주지역개발']

,firmname,type,own,own2,ownername
35,신경주지역개발,1.0,0.754386,0.754386,태영건설
36,신경주지역개발,99.0,1.000000,1.000000,NaN


In [298]:
df[df['소속회사명'] == '신경주지역개발']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
214,비금융회사,신경주지역개발,동일인측,계열회사(국내+해외),계열회사(국내+해외),태영건설,25800,43%,-,-,25800.0,43%,1.0,1.0
215,비금융회사,신경주지역개발,동일인측,동일인측합계,동일인측합계,NaN,25800,43%,-,-,25800.0,43%,NaN,NaN
216,비금융회사,신경주지역개발,기타주주,기타,기타,NaN,34200,57%,-,-,34200.0,57%,99.0,99.0
217,비금융회사,신경주지역개발,총계,총계,총계,/,60000,100%,-,-,34200.0,100%,NaN,NaN


In [299]:
dfOwn.iloc[35, 2] = 25800 / 60000
dfOwn.iloc[35, 3] = 25800 / 60000
dfOwn.iloc[36, 2] = 34200 / 60000
dfOwn.iloc[36, 3] = 34200 / 60000
dfOwn[dfOwn['firmname'] == '신경주지역개발']

,firmname,type,own,own2,ownername
35,신경주지역개발,1.0,0.43,0.43,태영건설
36,신경주지역개발,99.0,0.57,0.57,NaN


In [300]:
dfOwn[dfOwn['firmname'] == '티에스케이코퍼레이션']

,firmname,type,own,own2,ownername
98,티에스케이코퍼레이션,1.0,0.626071,0.626071,티와이홀딩스
99,티에스케이코퍼레이션,99.0,0.747857,0.747857,NaN


In [301]:
dfOwn.iloc[99, 2] = 358357 / 958357
dfOwn.iloc[99, 3] = 358357 / 958357
dfOwn[dfOwn['firmname'] == '티에스케이코퍼레이션']

,firmname,type,own,own2,ownername
98,티에스케이코퍼레이션,1.0,0.626071,0.626071,티와이홀딩스
99,티에스케이코퍼레이션,99.0,0.373929,0.373929,NaN


In [302]:
dfOwn[dfOwn['firmname'] == '티와이신경주주택개발']

,firmname,type,own,own2,ownername
101,티와이신경주주택개발,1.0,1.0,1.0,태영건설
102,티와이신경주주택개발,99.0,1.0,1.0,NaN


In [ ]:
dfOwn.iloc[102, 2] = 0
dfOwn.iloc[102, 3] = 0
dfOwn[dfOwn['firmname'] == '티와이신경주주택개발']

In [304]:
dfOwn.to_excel("../data/ownership-input/2021/티와이홀딩스.xlsx", index=False)

## 삼성전자

In [329]:
df = pd.read_excel('../data/ownership-status/raw/2021/삼성전자.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [330]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
미라콤아이앤씨    1.0
Name: own2, dtype: float64

In [331]:
dfOwn.to_excel("../data/ownership-input/2021/삼성전자.xlsx", index=False)

## 에스케이

In [332]:
df = pd.read_excel('../data/ownership-status/raw/2021/SK.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [333]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
에스케이바이오텍       0.000000
에스케이에코플랜트      1.000048
에스케이텔레콤        0.999903
에스케이피아이씨글로벌    1.490000
Name: own2, dtype: float64

In [334]:
df[df['소속회사명'] == '에스케이바이오텍']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
1051,비금융회사,에스케이바이오텍,동일인측,동일인,동일인,NaN,-,-,-,-,NaN,-,0.0,0.0
1052,비금융회사,에스케이바이오텍,동일인측,친족,배우자/혈족1촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1053,비금융회사,에스케이바이오텍,동일인측,친족,혈족2~4촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1054,비금융회사,에스케이바이오텍,동일인측,친족,혈족5~6촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1055,비금융회사,에스케이바이오텍,동일인측,친족,인척4촌이내,NaN,-,-,-,-,NaN,-,NaN,NaN
1056,비금융회사,에스케이바이오텍,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0
1057,비금융회사,에스케이바이오텍,동일인측,비영리법인,비영리법인,NaN,-,-,-,-,NaN,-,NaN,0.0
1058,비금융회사,에스케이바이오텍,동일인측,등기된임원,등기된임원,NaN,-,-,-,-,NaN,-,NaN,0.0
1059,비금융회사,에스케이바이오텍,동일인측,자기주식,자기주식,NaN,-,-,-,-,NaN,-,NaN,0.0
1060,비금융회사,에스케이바이오텍,동일인측,계열회사(국내+해외),계열회사(국내+해외),SKPharmtecoInc.,22825332,100.00%,-,-,22825332.0,100.00%,1.0,1.0


In [335]:
dfOwn[dfOwn['firmname'] == '에스케이바이오텍']

,firmname,type,own,own2,ownername
177,에스케이바이오텍,0.0,0.0,0.0,NaN
178,에스케이바이오텍,1.0,NaN,NaN,SKPharmtecoInc.
179,에스케이바이오텍,99.0,0.0,0.0,NaN


In [337]:
dfOwn.iloc[178, 2] = 1
dfOwn.iloc[178, 3] = 1
dfOwn[dfOwn['firmname'] == '에스케이바이오텍']

,firmname,type,own,own2,ownername
177,에스케이바이오텍,0.0,0.0,0.0,NaN
178,에스케이바이오텍,1.0,1.0,1.0,SKPharmtecoInc.
179,에스케이바이오텍,99.0,0.0,0.0,NaN


In [338]:
df[df['소속회사명'] == '에스케이피아이씨글로벌']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
1577,비금융회사,에스케이피아이씨글로벌,동일인측,동일인,동일인,NaN,-,-,-,-,NaN,-,0.0,0.0
1578,비금융회사,에스케이피아이씨글로벌,동일인측,친족,배우자/혈족1촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1579,비금융회사,에스케이피아이씨글로벌,동일인측,친족,혈족2~4촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1580,비금융회사,에스케이피아이씨글로벌,동일인측,친족,혈족5~6촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1581,비금융회사,에스케이피아이씨글로벌,동일인측,친족,인척4촌이내,NaN,-,-,-,-,NaN,-,NaN,NaN
1582,비금융회사,에스케이피아이씨글로벌,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0
1583,비금융회사,에스케이피아이씨글로벌,동일인측,비영리법인,비영리법인,NaN,-,-,-,-,NaN,-,NaN,0.0
1584,비금융회사,에스케이피아이씨글로벌,동일인측,등기된임원,등기된임원,NaN,-,-,-,-,NaN,-,NaN,0.0
1585,비금융회사,에스케이피아이씨글로벌,동일인측,자기주식,자기주식,NaN,-,-,-,-,NaN,-,NaN,0.0
1586,비금융회사,에스케이피아이씨글로벌,동일인측,계열회사(국내+해외),계열회사(국내+해외),에스케이씨,1020000,51.00%,-,-,1020000.0,51.00%,1.0,1.0


In [340]:
dfOwn[dfOwn['firmname'] == '에스케이피아이씨글로벌']

,firmname,type,own,own2,ownername
284,에스케이피아이씨글로벌,0.0,0.00,0.00,NaN
285,에스케이피아이씨글로벌,1.0,0.51,0.51,에스케이씨
286,에스케이피아이씨글로벌,99.0,0.98,0.98,NaN


In [341]:
dfOwn.iloc[286, 2] = 0.49
dfOwn.iloc[286, 3] = 0.49
dfOwn[dfOwn['firmname'] == '에스케이피아이씨글로벌']

,firmname,type,own,own2,ownername
284,에스케이피아이씨글로벌,0.0,0.00,0.00,NaN
285,에스케이피아이씨글로벌,1.0,0.51,0.51,에스케이씨
286,에스케이피아이씨글로벌,99.0,0.49,0.49,NaN
